## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [3]:
(x, y), (x_test, y_test) = datasets.mnist.load_data()
print(x.shape)

(60000, 28, 28)


## 建立模型

In [4]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28, 1000], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 1000],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[1000], dtype=tf.float32, initial_value=tf.zeros(1000))
        self.W2 = tf.Variable(shape=[1000, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[1000, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        newx = tf.reshape(x, shape=[-1, 28*28])
        hidden = tf.nn.relu(tf.matmul(newx, self.W1) + self.b1)
        logits = tf.matmul(hidden, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3726969 ; accuracy 0.1543
epoch 1 : loss 2.3245313 ; accuracy 0.15898333
epoch 2 : loss 2.2813265 ; accuracy 0.16463333
epoch 3 : loss 2.2421443 ; accuracy 0.17228334
epoch 4 : loss 2.2062736 ; accuracy 0.1823
epoch 5 : loss 2.1731582 ; accuracy 0.1949
epoch 6 : loss 2.1423507 ; accuracy 0.21128333
epoch 7 : loss 2.1134856 ; accuracy 0.22973333
epoch 8 : loss 2.086263 ; accuracy 0.25135
epoch 9 : loss 2.0604336 ; accuracy 0.27336666
epoch 10 : loss 2.0357914 ; accuracy 0.29526666
epoch 11 : loss 2.0121667 ; accuracy 0.3162
epoch 12 : loss 1.989419 ; accuracy 0.33748335
epoch 13 : loss 1.9674335 ; accuracy 0.35803333
epoch 14 : loss 1.9461185 ; accuracy 0.37865
epoch 15 : loss 1.9253979 ; accuracy 0.39816666
epoch 16 : loss 1.905213 ; accuracy 0.41636667
epoch 17 : loss 1.8855146 ; accuracy 0.43435
epoch 18 : loss 1.8662637 ; accuracy 0.45031667
epoch 19 : loss 1.8474301 ; accuracy 0.4664
epoch 20 : loss 1.8289868 ; accuracy 0.47956666
epoch 21 : loss 1.8109125 ; accura